## PDI - Phuture DeFi Index - Rebalancing Data January 2024

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
date = '2024-01-01'
min_mcap =  2.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                'ethereum',
                                '0x632806BF5c8f062932Dd121244c9fbe7becb8B48',
                                'pdi_benchmark_data',
                                'pdi_liquidities',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    'ethereum',
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['balancer','liquity','sushi'],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink']
    )
liquidity

Dropping dydx because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
lido-dao,0.309723,3.108076e-01,0.003501,ethereum
convex-finance,0.305171,3.049520e-01,-0.000716,ethereum
aave,0.009668,9.658818e-03,-0.000965,ethereum
rocket-pool,0.034699,3.453327e-02,-0.004787,ethereum
ribbon-finance,3.069493,3.054247e+00,-0.004967,ethereum
maker,0.000548,5.420186e-04,-0.011416,ethereum
liquity,0.768713,7.595917e-01,-0.011866,ethereum
compound-governance-token,0.018535,1.827858e-02,-0.013844,ethereum
dydx,0.374471,3.689998e-01,-0.014610,ethereum


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
uniswap,UNI,5023476929,0.300000,77,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,ethereum
lido-dao,LDO,2843290427,0.183190,47,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
maker,MKR,1681608649,0.108344,28,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
aave,AAVE,1517431267,0.097766,25,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
havven,SNX,1182591104,0.076193,20,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,ethereum
frax-share,FXS,672157808,0.043306,12,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
curve-dao-token,CRV,619424182,0.039909,10,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
rocket-pool,RPL,572467464,0.036883,9,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
1inch,1INCH,517673567,0.033353,8,0x111111111117dc0aa78b770fa6a738034120c302,ethereum
compound-governance-token,COMP,365464240,0.023546,6,0xc00e94cb662c3520282e6f5717214004a7f26888,ethereum


## Contract Output

In [5]:
methodology_class.output_for_contract()

0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0x111111111117dc0aa78b770fa6a738034120c302,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6123b0049f904d730db3c36a31167d9d4121fa6b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x6dea81c8171d0ba574754ef6f8b412f2ed88c54d,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
0,8,77,12,4,47,4,3,2,25,28,6,20,9,10
